In [607]:
%matplotlib inline

In [833]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [834]:
hybrid = gpd.read_file('/Users/michaelfoley/Google Drive/My Drive/Subnational_Yield_Database/boundaries/country/IND/2016_2023_hybrid_boundary.shp')
ag_stats = pd.read_csv('/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/ag_stats_final_nozeros_031025.csv')

In [835]:
import chardet
with open('/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/ag_stats_final_031025.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
print(f"Detected encoding: {result['encoding']} with confidence {result['confidence']}")

In [836]:
ag_stats

In [837]:
hybrid

In [838]:
hybrid.plot()

# Modifications
- drop columns we don't care about
    -columns to keep
        -id, boundary, name, wikidata, wikipedia, merged_dis, combined, group_numb, multidist, change_yea
- make a separate column that assigns the state to each district
- check to see why number of district names is different than the number of district names in the relationship table in 2016
- apply state abbreviation to every district

Also check to see why the number of district names is different than the number of district names in the relationship table in 2016. Next

In [839]:
filtered_hybrid = hybrid[['id', 'boundary', 'name', 'STATE', 'name_state', 'wikidata', 'wikipedia', 'merged_dis', 'combined', 'group_numb', 'multidistr', 'change_yea', 'geometry']]

In [840]:
filtered_hybrid

In [841]:
ag_stats

In [842]:
maize_test_2019 = ag_stats[(ag_stats['Year'] == 2019) & (ag_stats['Source crop'] == 'Maize')]
maize_test_2021 = ag_stats[(ag_stats['Year'] == 2021) & (ag_stats['Source crop'] == 'Maize')]

In [843]:
maize_test_2019.value_counts('Season')

In [844]:
maize_test_2021.value_counts('Season')

In [845]:
np.quantile(maize_test_2019[maize_test_2019['Season'] == 'Autumn']['Yield: MT/ha'], [0, 0.25, 0.5, 0.75, 1])

In [846]:
np.quantile(maize_test_2021[maize_test_2021['Season'] == 'Autumn']['Yield: MT/ha'], [0, 0.25, 0.5, 0.75, 1])

In [847]:
maize_test_2021[maize_test_2021['Season'] == 'Autumn']

In [848]:
len(maize_test_2019[maize_test_2019['Season'] == 'Autumn']['Yield: MT/ha'])

In [849]:
plt.hist(maize_test_2019[maize_test_2019['Season'] == 'Autumn']['Yield: MT/ha'], bins=20)

In [850]:
plt.hist(maize_test_2021[maize_test_2021['Season'] == 'Autumn']['Yield: MT/ha'], bins=20)

In [851]:
np.count_nonzero(maize_test_2019[(maize_test_2019['Season'] == 'Kharif')]['Yield: MT/ha'])

In [852]:
np.count_nonzero(maize_test_2021[(maize_test_2021['Season'] == 'Kharif')]['Yield: MT/ha'])

# Filter change boundary names to help aggregation

In [853]:
#mapping hybrid to crops
#decide what to do with Hanamkonda - left as Warangal Urban (TG) for now as that is how it is represented in the data, so I have some duplicates because of it.
#maybe rename during data aggregation phase? Same with prayagraj - look into. And kalaburagi, though both prayagraj and kalaburagi were changed before 2016 so they should be fine.
#missed Shi Yomi division in 2018 from West Siang
hybrid_mapping = {
    'Prayagraj (UP)': 'Allahabad (UP)', 
    'Aurangabad (MH) (MH)': 'Aurangabad (MH)',
    'Budgam (JK)': 'Badgam (JK)',
    'Bandipora (JK)': 'Bandipore (JK)',
    'Baramulla (JK)': 'Baramula (JK)', 
    'Ballari (KA)': 'Bellary (KA)',
    'Charkhi Dadri (HR)': 'Charki Dadri (HR)', #check what this one became
    'Siddarth Nagar (UP)': 'Siddharth Nagar (UP)',
    'Kalaburagi (KA)': 'Gulbarga (KA)',
    'South Sikkhim (SK)': 'South Sikkim (SK)',
    'Shivamogga (KA)': 'Shimoga (KA)',
}

merged_dis_mapping = {
    'Bhiwani (HR), Charkhi Dadri (HR)': 'Bhiwani (HR), Charki Dadri (HR)', #check what this one became
}

ag_mapping = {
    'Siaha (MZ)': 'Saiha (MZ)',
}

filtered_hybrid['name_state'] = filtered_hybrid['name_state'].replace(hybrid_mapping)
filtered_hybrid['merged_dis'] = filtered_hybrid['merged_dis'].replace(merged_dis_mapping)
ag_stats['Admin 2'] = ag_stats['Admin 2'].replace(ag_mapping)

# Now we try to aggregate all of the data. To do so, we will cycle through each of the groups for each year and aggregate any name matches

In [854]:
# Filter crop data for years 2016 onwards
hybrid_stats = ag_stats[ag_stats['Year'] >= 2016]

In [855]:
def aggregate_extra_value(s):
    """
    Returns the first non-null value if all non-null values in s are identical.
    If there are conflicting non-null values, returns the first value.
    """
    unique_vals = s.dropna().unique()
    if len(unique_vals) == 1:
        return unique_vals[0]
    elif len(unique_vals) == 0:
        return None
    else:
        return unique_vals[0]

def aggregate_extra_flag(s):
    """
    Returns True if there is more than one unique non-null value in s,
    indicating that the values are inconsistent.
    """
    unique_vals = s.dropna().unique()
    return len(unique_vals) > 1

In [856]:
def aggregate_data_for_polygon(districts, crop_df, polygon_name):
    """
    Aggregates crop data for a list of districts belonging to one polygon.
    
    Grouping is done by Year, CropType, and Season.
      - Sums AreaHarvested and QuantityProduced.
      - Computes weighted yield (using AreaHarvested as the weight).
      
    For extra (metadata) columns, we aggregate by taking the first non-null value,
    and we also compute a flag (e.g., Source_flag) that is True if there are conflicting values.
    
    For each aggregated group where any flag is True, we assign a unique agg_group_id and
    extract the original rows for that group for further inspection.
    
    Returns a tuple of:
      - The aggregated DataFrame (with an added 'agg_group_id' column for flagged groups).
      - A DataFrame of flagged rows from the original crop data, each with an 'agg_group_id' that connects it to the aggregated row.
      - A list of indices from crop_df that were aggregated.
    """
    global group_counter
    
    # Filter crop data for matching districts
    subset = crop_df[crop_df['Admin 2'].isin(districts)].copy()
    if subset.empty:
        return None, pd.DataFrame(), []
    
    # Record indices being aggregated (for later removal from df_crop)
    aggregated_indices = subset.index.tolist()
    
    # Helper column for weighted yield
    subset['yield_weighted'] = subset['Yield: MT/ha'] * subset['Area Harvested: ha']
    
    # Grouping keys: only aggregate same Year, CropType, Season
    group_keys = ['Year', 'Source crop', 'Season']
    
    # Numeric columns we aggregate
    numeric_cols = ['Area Harvested: ha', 'Quantity Produced: MT', 'Yield: MT/ha', 'yield_weighted']
    # Extra (metadata) columns: exclude grouping keys, numeric columns, and District identifier
    extra_cols = [col for col in subset.columns if col not in group_keys + numeric_cols + ['Admin 2']]
    
    # Group the subset by the grouping keys
    grouped = subset.groupby(group_keys, as_index=False)
    
    # Aggregate numeric values
    agg = grouped.agg(
        area_harvested=('Area Harvested: ha', 'sum'),
        quantity_produced=('Quantity Produced: MT', 'sum'),
        yield_weighted_sum=('yield_weighted', 'sum')
    )
    # Compute weighted yield for each group
    agg['weighted_yield'] = agg['yield_weighted_sum'] / agg['area_harvested']
    agg.drop(columns='yield_weighted_sum', inplace=True)
    
    # Aggregate extra columns and compute flags
    if extra_cols:
        agg_extra = grouped.agg({col: lambda s: aggregate_extra_value(s) for col in extra_cols}).reset_index()
        agg_flags = grouped.agg({col: lambda s: aggregate_extra_flag(s) for col in extra_cols}).reset_index()
        # Rename flag columns to add _flag suffix
        agg_flags.rename(columns={col: f"{col}_flag" for col in extra_cols}, inplace=True)
        extra = pd.merge(agg_extra, agg_flags, on=group_keys)
        # Merge extra aggregated info into agg
        agg = pd.merge(agg, extra, on=group_keys, how='left')
    
    # Add polygon_name to aggregated data
    agg['polygon_name'] = polygon_name
    
    # Create a column to hold the aggregated group ID (for flagged groups)
    agg['agg_group_id'] = None
    flagged_rows = []  # List to collect flagged original rows
    
    # Identify which columns are the flag columns
    flag_columns = [col for col in agg.columns if col.endswith('_flag')]
    
    # For each aggregated group, if any flag is True, record the group ID and extract the original rows
    for i, row in agg.iterrows():
        # Check if any flag is True in this group
        flags = [row[flag] for flag in flag_columns if flag in row]
        if any(flags):
            # Assign a unique group id
            group_id = f"G{group_counter}"
            group_counter += 1
            # Set the group id for the aggregated row
            agg.at[i, 'agg_group_id'] = group_id
            # Extract all original rows from subset matching this group
            # (match on each grouping key: Year, CropType, Season)
            condition = True
            for key in group_keys:
                condition = condition & (subset[key] == row[key])
            flagged_group = subset[condition].copy()
            flagged_group['agg_group_id'] = group_id
            flagged_group['polygon_name'] = polygon_name
            flagged_rows.append(flagged_group)
    
    if flagged_rows:
        flagged_df = pd.concat(flagged_rows, ignore_index=True)
    else:
        flagged_df = pd.DataFrame()
    
    return agg, flagged_df, aggregated_indices



In [857]:
# Global counter to assign unique aggregated group IDs for flagged groups
group_counter = 1

# Lists to collect results across all polygons
aggregated_results = []
flagged_results = []
all_aggregated_indices = set()

# Iterate over each polygon in the shapefile
for idx, poly in filtered_hybrid.iterrows():
    district_names = poly.get('merged_dis', None)
    if not district_names:
        print(f"Skipping polygon {poly.get('id', idx)} because 'merged_dis' is missing.")
        continue
    else:
        print(f"Aggregating {poly.get('merged_dis')}")

    district_list = [name.strip() for name in district_names.split(',')]
    polygon_name = poly.get('name_state', poly.get('id', idx))
    
    agg_data, flagged_data, indices = aggregate_data_for_polygon(district_list, hybrid_stats, polygon_name)
    if agg_data is not None:
        aggregated_results.append(agg_data)
        flagged_results.append(flagged_data)
        all_aggregated_indices.update(indices)

# Combine aggregated results from all polygons into one DataFrame
if aggregated_results:
    agg_df = pd.concat(aggregated_results, ignore_index=True)
else:
    agg_df = pd.DataFrame()

# Combine all flagged rows from polygons into one DataFrame
if flagged_results:
    flagged_df = pd.concat(flagged_results, ignore_index=True)
else:
    flagged_df = pd.DataFrame()

# Remove the aggregated rows from the original crop data
remaining_crop = hybrid_stats.drop(index=all_aggregated_indices).copy()

# Optionally add a polygon_name column to remaining rows if desired
if 'polygon_name' not in remaining_crop.columns:
    remaining_crop['polygon_name'] = None

# Create final DataFrame: aggregated rows followed by non-aggregated rows.
final_df = pd.concat([agg_df, remaining_crop], ignore_index=True)

# Now you have:
#  - agg_df: Aggregated crop data (with agg_group_id for flagged groups)
#  - flagged_df: Original rows (with agg_group_id) that had inconsistent extra values
#  - final_df: Full crop data with aggregated rows replacing original ones, ready to merge back into the shapefile.

print("Aggregated DataFrame (agg_df):")
print(agg_df.head())

print("\nFlagged Rows for Further Inspection (flagged_df):")
print(flagged_df.head())

print("\nFinal DataFrame (final_df):")
print(final_df.head())

In [858]:
agg_df

In [859]:
flagged_df

In [860]:
final_df['Area Harvested: ha'].isna().sum()

In [861]:
final_df['area_harvested'].notna().sum()

Great. All we need to do now is merge those columns, apply consistent formating to yields to ensure no super long decimals, remove flag columns, and test compared to icrisat yields from 2016 visually. Also need to remove:
- Budgam (JK)

In [862]:
final_df = final_df[final_df['Admin 2'] != 'Budgam (JK)']

# Modifications to main data table
 - Need to fix Rabi season in main data table - should go to 1/31/[next year] [DONE]
 - Remove zeros [DONE]
 - Standardize source year [DONE]
 - Summer Ballari start and end dates 2022 need year fixed (currently /23)
 - fix source document (can do later)

# Modifications to boundary file
 - every name should be stored with it's appropriate state name to avoid confusion (particularly Bilaspur). This is tough though when states change. [DONE]
 - Must clarify bijapur, hamirpur, and pratapgarh [DONE]
 

In [863]:
final_df

In [864]:
#Try merging aggregated area harvested with original area harvested
final_df['Area Harvested: ha'] = final_df['Area Harvested: ha'].fillna(final_df['area_harvested'])
final_df['Quantity Produced: MT'] = final_df['Quantity Produced: MT'].fillna(final_df['quantity_produced'])
final_df['Yield: MT/ha'] = final_df['Yield: MT/ha'].fillna(final_df['weighted_yield'])

In [865]:
for column in final_df.columns:
    if 'flag' in column:
        final_df.drop(columns=column, inplace=True)

In [866]:
final_df['Admin 2'].isna().sum()

In [867]:
final_df['polygon_name'].notna().sum()

# Merge polygon_name to Admin 2

In [868]:
final_df['Admin 2'] = final_df['Admin 2'].fillna(final_df['polygon_name'])

In [869]:
column_sort = ['Data Source Organization',	'Data Source Document',	'Publication Name', 'Survey Type', 'Country', 'Zone', 'FNID', 
'Admin 1', 'Admin 2', 'Admin 3', 'Year', 'Start Period', 'Season', 'End Period', 'Crop', 'Dominant Production System', 
'Area Planted: ha', 'Area Harvested: ha', 'Yield: MT/ha', 'Quantity Produced: MT', 'Contributions by', 'Source crop', 'DNL_Source_year']

In [870]:
final_df = final_df[column_sort]

In [ ]:
final_df.to_csv('/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/2016_2023_hybrid_stats.csv')

# Merge hybrid boundary and this data


In [897]:
filtered_hybrid.to_file('/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/2016_2023_hybrid_boundary.shp')

In [873]:
merged_ourdata = final_df.merge(filtered_hybrid, left_on='Admin 2', right_on='name_state', how='left')
merged_ourdata = gpd.GeoDataFrame(merged_ourdata, geometry='geometry')

# Check if merge was successful
print(f"Original CSV DataFrame shape: {final_df.shape}")
print(f"Merged GeoDataFrame shape: {merged_ourdata.shape}")
print(f"Number of rows with geometry: {merged_ourdata.geometry.notna().sum()}")

In [874]:
#merged_ourdata.to_file('/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/hybrid_data_2016_2023.shp')

In [875]:
# Count rows missing geometry
missing_geometry = merged_ourdata[merged_ourdata.geometry.isna()]
print(f"Number of rows missing geometry: {len(missing_geometry)}")

# Look at some examples of rows missing geometry
sorted(missing_geometry['Admin 2'].unique())

In [876]:
missing_geometry['Admin 2'].value_counts()

In [885]:
def overlap_check(old_df, final_df):
    # Check for overlap
    check_columns = ['Year', 'Source crop', 'Season', 'Area Harvested: ha', 'Yield: MT/ha', 'Quantity Produced: MT']
    
    # Convert year in test dataframe (already float for others)
    old_df['Year'] = old_df['Year'].astype(int)
    old_df['Area Harvested: ha'] = pd.to_numeric(old_df['Area Harvested: ha'], errors='coerce').round(2)
    old_df['Yield: MT/ha'] = pd.to_numeric(old_df['Yield: MT/ha'], errors='coerce').round(2)
    old_df['Quantity Produced: MT'] = pd.to_numeric(old_df['Quantity Produced: MT'], errors='coerce').round(2)

    # Now try the merge
    matches = final_df.merge(
        old_df, 
        on=check_columns,
        indicator=True,
        how='left'
    )

    # Count matches and non-matches
    match_stats = matches['_merge'].value_counts()

    print("\nResults:")
    print(f"Total rows in test dataset: {len(final_df)}")
    print(f"Number of matching rows: {sum(matches['_merge'] == 'both')}")
    print(f"Number of non-matching rows: {sum(matches['_merge'] == 'left_only')}")
    print(f"Percentage of test rows found in main dataset: {(sum(matches['_merge'] == 'both') / len(final_df)) * 100:.2f}%")

In [894]:
shivamogga = merged_ourdata[merged_ourdata['Admin 2'] == 'Shivamogga (KA)']
shimoga = merged_ourdata[merged_ourdata['Admin 2'] == 'Shimoga (KA)']
kadapa = merged_ourdata[merged_ourdata['Admin 2'] == 'Kalaburagi (KA)']
ysr_kadapa = merged_ourdata[merged_ourdata['Admin 2'] == 'Gulbarga (KA)']

In [896]:
overlap_check(kadapa, ysr_kadapa)

In [877]:
missing_geometry.columns

In [878]:
sorted(hybrid['name_state'].unique())

In [879]:
merged_2017_rice = merged_ourdata[(merged_ourdata['Year'] == 2017) & (merged_ourdata['Source crop'] == 'Rice')]

In [880]:
merged_2017_rice

In [881]:
merged_2017_rice

In [882]:
missing_geometry

In [883]:
valid_names_from_final = final_df['Admin 2'].dropna().unique()
filtered_hybrid_new = filtered_hybrid[filtered_hybrid['name_state'].isin(valid_names_from_final)]

In [884]:
fig, ax = plt.subplots(1, 1, figsize=(20,20))

merged_2017_rice.plot(ax = ax,
    column='Yield: MT/ha',
    legend=True,
    vmin = 0,
    vmax = 4)


filtered_hybrid_new.boundary.plot(ax = ax,
    linewidth=1.0, 
    edgecolor='black'
)

# Now lets make a map to compare ICRISAT yield data to our data 

In [170]:
icrisat_path = '/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/icrisat_apportioned/'
shape_path = icrisat_path + 'icrisat_boundary_match.shp'
data_path = icrisat_path + 'ICRISAT-District Level Data_Apportioned.csv'

# Read the shapefile of district boundaries
icrisat_districts = gpd.read_file(shape_path)

# Read the CSV containing crop data (which includes columns like 'district_name', 'year', 'crop', 'area', 'production', 'yield')
icrisat_data = pd.read_csv(data_path)

In [171]:
# First, create a temporary dataframe with only the columns we need for joining
# plus the geometry from the shapefile
icrisat_lookup = icrisat_districts[['NAME_1', 'Dist_Name', 'geometry']]

icrisat_lookup = icrisat_lookup.rename(columns={
    'NAME_1': 'State Name',
    'Dist_Name': 'Dist Name'
})

In [172]:
# Now merge the dataframes
icrisat_df = icrisat_data.merge(
    icrisat_lookup,
    on=['State Name', 'Dist Name'],
    how='left'  # Use 'left' to keep all spatial features, 'inner' for only matching rows
)

# Convert the result to a GeoDataFrame
icrisat_gdf = gpd.GeoDataFrame(icrisat_df, geometry='geometry')

# Check if merge was successful
print(f"Original CSV DataFrame shape: {icrisat_data.shape}")
print(f"Merged GeoDataFrame shape: {icrisat_gdf.shape}")
print(f"Number of rows with geometry: {icrisat_gdf.geometry.notna().sum()}")

In [173]:
# Count rows missing geometry
missing_geometry = icrisat_gdf[icrisat_gdf.geometry.isna()]
print(f"Number of rows missing geometry: {len(missing_geometry)}")

# Look at some examples of rows missing geometry
missing_geometry[['State Name', 'Dist Name']]

In [174]:
icrisat_2016 = icrisat_gdf[icrisat_gdf['Year'] == 2016]
icrisat_2016.plot(column = 'RICE YIELD (Kg per ha)',
    vmin = 0,
    vmax = 4000)

In [175]:
final_2016 = final_df[(final_df['Year'] == 2016) & (final_df['Source crop'] == 'Rice')]